<a href="https://colab.research.google.com/github/KashyretsIvanna/ml/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colaboratory!%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Лабораторна робота 2**

> Добавить блок с цитатой


ПРОГНОЗУВАННЯ НА ОСНОВІ КЛАСИФІКАТОРА ДЕРЕВА РІШЕНЬ

Виконала студентка КП-41мп

Каширець Іванна

2025












# Частина 1

**Завдання:**

Потрібно створити класифікатор дерева рішень, який буде навчатися на наборі даних із заданими мітками. Набір даних містить імена та дані кожного пасажира пасажирського лайнера «Титанік». Крім того, вказано деталі
подорожі пасажирів. На основі цих даних потрібно побудувати дерево рішень, яке ілюструє фактори, які сприяли виживаності або смертності під час
подорожі.

За наведеними вище даними сформулюйте питання про фактори
(атрибути), які сприяли тому, що пасажири вижили або загинули під час
катастрофи Титаніка.






### Крок 1. Завантаження даних

In [1]:
#import pandas
import pandas as pd
#create a pandas dataframe called "training" from the titanic-train.csv file
training = pd.read_csv("titanic-train.csv")


2) Перевірте імпорт і прогляньте дані.

In [2]:
#Code cell 2
#verify the contents of the training dataframe using the pandas info() method.
#training.?
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  915 non-null    int64  
 1   Survived     915 non-null    int64  
 2   Pclass       915 non-null    int64  
 3   Name         915 non-null    object 
 4   Gender       915 non-null    object 
 5   Age          738 non-null    float64
 6   SibSp        915 non-null    int64  
 7   Parch        915 non-null    int64  
 8   Ticket       915 non-null    object 
 9   Fare         915 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     914 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 85.9+ KB


Висновок:

У представленому наборі даних є відсутні значення. Зокрема:

У стовпці Age відсутні 177 значень (915 - 738).
У стовпці Cabin відсутні 713 значень (915 - 202).
У стовпці Embarked відсутнє 1 значення (915 - 914).
Загальна кількість відсутніх значень у наборі даних становить 891.

In [4]:
#Code cell 3
#view the first few rows of the data
#data = pd.read_csv('clustering.csv')
training.head()

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,1,"Davidson, Mr. Thornton",male,31.0,1,0,F.C. 12750,52.0000,B71,S
1,2,0,3,"Asim, Mr. Adola",male,35.0,0,0,SOTON/O.Q. 3101310,7.0500,NaN,S
2,3,0,3,"Nankoff, Mr. Minko",male,NaN,0,0,349218,7.8958,NaN,S
3,4,0,1,"Thayer, Mr. John Borland",male,49.0,1,1,17421,110.8833,C68,C
4,5,0,3,"Strandberg, Miss. Ida Sofia",female,22.0,0,0,7553,9.8375,NaN,S


### Крок 2. Підготуйте дані для побудови моделі дерева рішень.

Значення змінної статі Gender необхідно перетворити в
числові представлення: 0 буде використовуватися для позначення «чоловіків»,
а 1 – «жінок».

In [6]:
#code cell 4
training["Gender"] = training["Gender"].apply(lambda toLabel: 0 if toLabel == 'male' else 1)

Переконаймося, що змінна Gender була змінена.

In [7]:
#code cell 5
#view the first few rows of the data again
training.head()

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,1,"Davidson, Mr. Thornton",0,31.0,1,0,F.C. 12750,52.0000,B71,S
1,2,0,3,"Asim, Mr. Adola",0,35.0,0,0,SOTON/O.Q. 3101310,7.0500,NaN,S
2,3,0,3,"Nankoff, Mr. Minko",0,NaN,0,0,349218,7.8958,NaN,S
3,4,0,1,"Thayer, Mr. John Borland",0,49.0,1,1,17421,110.8833,C68,C
4,5,0,3,"Strandberg, Miss. Ida Sofia",1,22.0,0,0,7553,9.8375,NaN,S


Отже, значення були замінені на 0 та 1

Звернімося до відсутніх значень у наборі даних.

In [8]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  915 non-null    int64  
 1   Survived     915 non-null    int64  
 2   Pclass       915 non-null    int64  
 3   Name         915 non-null    object 
 4   Gender       915 non-null    int64  
 5   Age          738 non-null    float64
 6   SibSp        915 non-null    int64  
 7   Parch        915 non-null    int64  
 8   Ticket       915 non-null    object 
 9   Fare         915 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     914 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 85.9+ KB


Вік має 915-738 пропущених значень

Заповнимо пропущені значення середніми:

In [9]:
#code cell 6
training["Age"].fillna(training["Age"].mean(), inplace=True)

<ipython-input-9-de089a03bd2b>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  training["Age"].fillna(training["Age"].mean(), inplace=True)


Переконайтеся, що значення були замінені.

In [10]:
#verify that the missing values for the age variable have been eliminated.
training.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  915 non-null    int64  
 1   Survived     915 non-null    int64  
 2   Pclass       915 non-null    int64  
 3   Name         915 non-null    object 
 4   Gender       915 non-null    int64  
 5   Age          915 non-null    float64
 6   SibSp        915 non-null    int64  
 7   Parch        915 non-null    int64  
 8   Ticket       915 non-null    object 
 9   Fare         915 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     914 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 85.9+ KB


Кількість не нульових значеннь дорівнює кількості записів, отже всі пропущені значення були заповнені

Яке значення було використано для заміни відсутніх значень віку? Було використано mean  - середнє значення

### Крок 3. Навчіть і оцініть модель класифікатора дерева рішень.

а) Створіть об'єкт масиву зі змінною, яка буде цільовою для моделі.

In [ ]:
#code cell 8
#create the array for the target values
y_target = training["Survived"].values

б) Створіть масив значень, які будуть вхідними для моделі.
Змінна X_input містить значення для всіх ознак,
які модель використовуватиме, щоб дізнатися, як виконувати класифікацію.
Після навчання моделі потрібно використати цю змінну, щоб призначити
мітки набору тестових даних.


In [ ]:
columns = ["Fare", "Pclass", "Gender", "Age", "SibSp"]
#create the variable to hold the features that the classifier will use
X_input = training[list(columns)].values

в) Створіть модель класифікатора дерева рішень.

Імпортуйте модуль дерева рішень з бібліотеки машинного навчання
sklearn. Створіть об'єкт класифікатора clf_train. Потім скористайтеся методом
fit() об’єкта класифікатора зі змінними X_input та y_target як параметрами,
щоб навчити модель.


# Висновок



# Питання для самоперевірки





In [ ]:
1. Що таке кластеризація? Які властивості кластерів ви знаєте?
Кластеризація – це техніка машинного навчання, яка включає групування точок даних у множини,
які називаються кластерами.  Враховуючи набір точок даних, ми можемо використовувати алгоритм
кластеризації, щоб класифікувати кожну точку даних у певну групу. Точки  даних, які входять до однієї групи,
повинні мати подібні властивості та/або  особливості, тоді як точки даних у різних групах повинні мати дуже різні
властивості та/або особливості.

2. Кластеризація відноситься до задач машинного навчання без нагляду
(Unsupervised learning) чи це контроловане навчання (Supervised learning)?
Кластеризація є методом неконтрольованого  навчання (Unsupervised learning) і є поширеною технікою для аналізу
статистичних даних.


3. Які методи та алгоритми кластеризації ви знаєте?
  1. Partitioning methods (Методи розбиття)
  k-Means – поділяє дані на k кластерів, мінімізуючи відстань між точками та центроїдами.
  k-Medoids – схожий на k-Means, але вибирає реальні точки як центроїди, що підвищує стійкість до викидів.
  k-Modes – розширює k-Means для категоріальних даних.
  Fuzzy C-Means  – дозволяє точці належати до кількох кластерів із певною ймовірністю.
  PAM , CLARA , CLARANS  – варіації k-Medoids для обробки великих наборів даних.

  2. Hierarchical methods (Ієрархічні методи)
  Divisive  – розпочинається з одного великого кластера і поступово розділяє його.
  Agglomerative (AGNES, BIRCH, CURE, ROCK, Chameleon) – навпаки, починає з окремих точок і поступово об’єднує їх у кластери.

  3. Density-based methods (Методи, що базуються на щільності)
  DBSCAN  – знаходить кластери довільної форми на основі щільності точок.
  OPTICS – розширена версія DBSCAN, краще працює з кластерами різної щільності.
  STING , CLIQUE – використовують сіткове подання простору для швидкої кластеризації.
  DENCLUE, Wave Cluster – методи, що використовують оцінку щільності функції розподілу даних.

  4. Graph-based methods (Графові методи кластеризації)
  MST Clustering – базується на мінімальному остовному дереві (MST).
  OPOSSUM , SNN Similarity Clustering  – методи, що використовують схожість між сусідами в графах.

  5. Model-based clustering (Модельні методи кластеризації)
  EM Algorithm  – оцінює ймовірності приналежності точки до кластерів (використовується у Gaussian Mixture Models).
  Auto Class  – підхід, що використовує байєсівські методи для кластеризації.
  COBWEB  – ієрархічний підхід для поступового розбиття даних.
  ANN Clustering  – методи, засновані на штучних нейронних мережах.

4. Наведіть приклади ієрархічних, щільнісних та ітеративних методів
кластеризації, їх переваги та недоліки.
Ітеративні:
  1) Алгоритм k-середніх:
    Переваги алгоритму k- means:
    − простота та швидкість використання.
    Недоліки алгоритму k- means:
    − алгоритм дуже чутливий до викидів (outliers), які можуть  спотворювати середнє.
    Можливим вирішенням цієї проблеми є  використання модифікації алгоритму – алгоритм k-медіани;
    − алгоритм може повільно працювати на великих наборах даних.
  2) Метод найближчого сусіда:
    Переваги:
    Простота реалізації та інтуїтивне розуміння.
    Гнучкість – не вимагає припущень про розподіл даних.
    Може ефективно працювати з нелінійними межами між класами.
    Використовує всю навчальну вибірку, що робить його універсальним для різних задач.
    Недоліки:
    Висока обчислювальна складність при великих обсягах даних (O(n) на один запит).
    Чутливість до вибору метрики відстані та параметра k.
    Погано працює у випадках, коли класи мають неоднорідну щільність або складну структуру.
    Вразливий до вибору навчальних даних і може давати хибні результати при наявності шуму.
  3) Алгоритм ЕМ
    Алгоритм EM простий в реалізації, не чутливий до ізольованих
    об'єктів  і швидко сходиться при вдалій ініціалізації. Однак він вимагає для
    ініціалізації  кількості кластерів k, що передбачає наявність апріорних знань про дані.
    Крім  того, при невдалій ініціалізації збіжність алгоритму може виявитися  повільною або може бути отриманий
    неякісний результат.
  4) Алгоритм PAM:
    Алгоритм менш чутливий до шумів і викидів даних, ніж алгоритм  k-means,
    оскільки медіана менше піддається впливам викидів. PAM  ефективний для невеликих наборів даних,
    його не бажано використовувати  для великих наборів даних.
Ієрархічні:
  1)Алгоритм BIRCH:
    Завдяки узагальненому вигляду кластерів, швидкість кластеризації  збільшується,
    алгоритм BIRCH (Balanced Iterative Reducing and Clustering  using Hierarchies) при цьому володіє великим
    масштабуванням.
  2)Алгоритм WaveCluster:
    Головні особливості  WaveCluster:
    − складність реалізації;
    − алгоритм може виявляти кластери довільних форм;
    − алгоритм не чутливий до шумів;
    − алгоритм може бути застосований тільки до даних низької  розмірності.
  3) Алгоритм CLARA:
    Для великих баз  даних цей алгоритм ефективніший, ніж алгоритм PAM.
Щільнісні:
  1)Алгоритм DBSCAN:
    Хоча, на відміну від методів розбиття, DBSCAN не  вимагає заздалегідь вказувати число одержуваних кластерів,
    виникне потреба  у вказівках значень параметрів ε і MinPts, які безпосередньо впливають на  результат кластеризації.
    Оптимальні значення цих параметрів складно  визначити, особливо для багатовимірних просторів даних.
  2)Алгоритм OPTICS:
    Його основна  ідея схожа на DBSCAN, але він вирішує один з основних недоліків DBSCAN –
    проблему визначення значущих кластерів в наборах даних різної щільності.


5. Які застосування кластеризації в реальних сценаріях ви знаєте?
  Сегментація клієнтів у маркетингу
  Виявлення аномалій у фінансах
  Аналіз текстів та групування документів
  Кластеризація біологічних даних (генів, білків)
  Кластеризація документів
  Сегментація зображень
6. Які метрики оцінки кластеризації ви знаєте?
Інерція,Індекс Данна
7. Що таке кластеризація K-Means? Як вибрати потрібну кількість кластерів
у K-Means?
  K-Means – це ітеративний алгоритм, що шукає K центрів кластерів.
  Вибір K:
  - Метод ліктя (Elbow Method)
  - Силуетний аналіз
8. Для чого використовується алгоритм K-Means ++ ?
  Використовується для покращення ініціалізації центрів кластерів, щоб уникнути локальних мінімумів.
9. Як реалізувати алгоритм кластеризації K-Means та алгоритм K-Means ++ у
Python?
  from sklearn.cluster import KMeans
  kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
  kmeans.fit(data)
  labels = kmeans.labels_
10. Для чого використовується стандартизація даних?
  Щоб привести дані до єдиного масштабу, що покращує роботу алгоритмів.


11. Як реалізувати у Python стандартизацію даних?
  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(data)
12. Для чого використовуються пакети Python: pandas, numpy, matplotlib,
random?
  pandas – обробка таблиць
  numpy – робота з масивами
  matplotlib – візуалізація
  random – генерація випадкових чисел
13. Які методи Python використовуются для прочитання файлу у форматі CSV
та перегляду перших п’яти рядків даних?
  import pandas as pd
  df = pd.read_csv("data.csv")
  print(df.head())
14. Як реалізувати у Python ініціалізацію центроїдів кластера та та
побудувати криву ліктя?
  SSE = []

  for cluster in range(1, 20):
      kmeans = KMeans(n_clusters=cluster, init='k-means++', n_init=10)
      kmeans.fit(data_scaled)
      SSE.append(kmeans.inertia_)

  # Converting the results into a dataframe and plotting them
  frame = pd.DataFrame({'Cluster': range(1, 20), 'SSE': SSE})

  plt.figure(figsize=(12, 6))
  plt.plot(frame['Cluster'], frame['SSE'], marker='o')
  plt.xlabel('Number of clusters')
  plt.ylabel('Inertia')
  plt.title('Elbow Method for Optimal Clusters')
  plt.show()
